<a href="https://colab.research.google.com/github/peppefdf/CSL_Gipuzkoa/blob/main/DragDrop_Markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [93]:
#%pip install osmnx
#%pip install dash
#%pip install dash_leaflet
#%pip install dash_bootstrap_components
#%pip install GDAL # install GDAL first and then rasterio (3.4.0?)
#%pip install rasterio
#%pip install peartree
#%pip install diskcache
###%pip install "dash[diskcache]"
#%pip install multiprocess
#%pip install psutil
#%pip install pandana

from dash import Dash
import dash_bootstrap_components as dbc
#import dash_html_components as html
from dash import html, callback_context
from dash import dcc, Output, Input, State, callback
import dash_leaflet as dl
import re

# Custom icon as per official docs https://leafletjs.com/examples/custom-icons/
custom_icon = dict(
    iconUrl='https://leafletjs.com/examples/custom-icons/leaf-green.png',
    shadowUrl='https://leafletjs.com/examples/custom-icons/leaf-shadow.png',
    iconSize=[38, 95],
    shadowSize=[50, 64],
    iconAnchor=[22, 94],
    shadowAnchor=[4, 62],
    popupAnchor=[-3, -76]
)
# Small example app.
positions = [(57, 11), (57, 10), (58, 10)]
markers = [dl.Marker(dl.Tooltip("double click on Marker after drag to update its position"), position=pos, id="marker{}".format(i), draggable=True) for i, pos in enumerate(positions)]

app = Dash(prevent_initial_callbacks=True)
app.layout = html.Div([
    html.Div(dl.Map([dl.TileLayer()]+
                     markers,
                     center=(57, 10.5), zoom=6, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})),
    html.Div(id='clickdata'),
    dcc.Store(id='internal-value', data=positions)
])

#@app.callback(Output("clickdata", "children"),
@app.callback([Output("clickdata", "children"),Output('internal-value','data'), Output('map','children')],
              State('internal-value','data'),
              [Input(marker.id, "dblclickData") for marker in markers])
def marker_click(St,*args):
    marker_id = callback_context.triggered[0]["prop_id"].split(".")[0]
    marker_lat = callback_context.triggered[0]['value']['latlng']['lat']
    marker_lon = callback_context.triggered[0]['value']['latlng']['lng']
    marker_s = callback_context.inputs
    temp = re.compile("([a-zA-Z]+)([0-9]+)")
    id = temp.match(marker_id).groups()[-1]
    #return "New position of {}".format(marker_id) + ": {}".format(marker_lat) + ", {}".format(marker_lon)
    St[int(id)] = (marker_lat,marker_lon)
    out = str(St[0][0]) + ', ' + str(St[0][1]) + '; '
    out = out + str(St[1][0]) + ', ' + str(St[1][1]) + '; '
    out = out + str(St[2][0]) + ', ' + str(St[2][1]) + '; '
    markers = [dl.Marker(dl.Tooltip("double click on Marker after drag to update its position"), position=pos, id="marker{}".format(i), draggable=True) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()]+
                     markers,
                     center=(57, 10.5), zoom=6, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [out,St, newMap]

if __name__ == '__main__':
    app.run_server(debug=True)

<IPython.core.display.Javascript object>